In [ ]:
'''01_setup_kafka_on_ubuntu'''
'''--------------------------------------------------------------------------------------'''
'''masuk ke dalam server ubuntu'''
'''jadikan user sebagai super user'''
sudo usermod -aG sudo super

'''update ubuntu'''
sudo apt update -y

'''install java'''
sudo apt install openjdk-11-jdk -y

'''cek versi java, jika muncul versi java berarti java sudah terinstall'''
java -version

'''selanjutnya install confluent'''
'''buat folder Downloads di dalam folder /home'''
sudo mkdir Downloads

'''masuk ke dalam folder Downloads'''
cd home/Downloads

'''download confluent'''
sudo curl -O https://packages.confluent.io/archive/7.5/confluent-7.5.2.tar.gz

'''pindahkan confluent ke dalam folder /opt/confluent'''
cd opt
mkdir confluent
cd confluent
sudo mv /home/Downloads/confluent-7.5.2 .

'''extract confluent, menggunakkan command berikut:'''
sudo tar -xvzf confluent-7.5.2.tar.gz
cd confluent-7.5.2

'''set environment variable CONFLUENT_HOME'''
'''masuk ke dalam folder confluent-7.5.2'''
cd /opt/confluent/confluent-7.5.2
'''lalu cari pathnya dengan command'''
pwd
'''akan muncul pathnya, copy pathnya, untuk diset ke dalam environment variable CONFLUENT_HOME'''
/opt/confluent/confluent-7.5.2

'''buka file .bashrc'''
vi ~/.bashrc

'''tambahkan baris berikut di dalam file .bashrc'''
export CONFLUENT_HOME=/opt/confluent/confluent-7.5.2
export PATH=$PATH:/usr/lib/jvm/java-11-openjdk-amd64/bin:/opt/confluent/confluent-7.5.2/bin

'''check environment variable CONFLUENT_HOME'''
echo $PATH
'''maka akan muncul path yang telah diset sebelumnya'''

'''check confluent status'''
confluent local services status
'''maka akan muncul status confluent, seharusnya statusnya masih belum running'''

'''start confluent'''
confluent local services kafka start

'''check lagi confluent status'''
'''kafka dan zookeeper sudah running'''

'''start schema-registry'''
confluent local services schema-registry start

'''check lagi confluent status'''
'''schema-registry sudah running'''

'''jalankan service kafka connect , menggunakan command berikut:'''
cd /opt/confluent/confluent-7.5.2/bin/
./connect-distributed ../etc/kafka/connect-distributed.properties

'''agar kafka connect berjalan otomatistis ketika server di restart, maka jalankan command berikut:'''
sudo vi /etc/systemd/system/kafka-connect.service
'''lalu copy paste script berikut ke dalam file kafka-connect.service'''
[Unit]
Description=Kafka Connect
After=network.target

[Service]
User=super
ExecStart=/opt/confluent/confluent-7.5.2/bin/connect-distributed /opt/confluent/confluent-7.5.2/etc/kafka/connect-distributed.properties
Restart=on-failure

[Install]
WantedBy=multi-user.target

'''lalu jalankan command berikut:'''
sudo systemctl daemon-reload
sudo systemctl enable kafka-connect
sudo systemctl start kafka-connect
sudo systemctl status kafka-connect

'''check lagi confluent status'''
confluent local services status
'''kafka connect sudah running'''

'''test kafka dengan cara hit API berikut:'''
curl -XGET http://localhost:8081

'''kalau muncul response seperti ini, berarti kafka sudah terinstall dengan baik'''
{}

'''install jq pretty print untuk mempermudah membaca response'''
'''pastikan python sudah terinstall'''
sudo apt-get install python3-pip

'''install pretty print'''
pip3 install jq

'''test kafka dengan cara hit API berikut:'''
curl -XGET http://localhost:8083/connector-plugins | jq

'''maka akan muncul response seperti berikut:'''
[
  {
    "class": "org.apache.kafka.connect.mirror.MirrorCheckpointConnector",
    "type": "source",
    "version": "7.5.2-ce"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorHeartbeatConnector",
    "type": "source",
    "version": "7.5.2-ce"
  },
  {
    "class": "org.apache.kafka.connect.mirror.MirrorSourceConnector",
    "type": "source",
    "version": "7.5.2-ce"
  }
]

'''jika belum muncul response: io.debezium.connector, maka install connector dengan cara berikut:'''
'''masuk ke dalam folder confluent-7.5.2'''
cd /opt/confluent/confluent-7.5.2
'''buat folder connectors'''
sudo mkdir connectors
cd connectors

'''download connector-connector yang kira-kira dibutuhkan'''
'''download connector mysql'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-mysql/2.4.1.Final/debezium-connector-mysql-2.4.1.Final-plugin.tar.gz
'''download connector postgresql'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-postgres/2.4.1.Final/debezium-connector-postgres-2.4.1.Final-plugin.tar.gz
'''download connector sqlserver'''
wget https://repo1.maven.org/maven2/io/debezium/debezium-connector-sqlserver/2.4.1.Final/debezium-connector-sqlserver-2.4.1.Final-plugin.tar.gz

'''extract connector, di dalam folder /opt/confluent/confluent-7.5.2/connectors'''
tar -xvzf debezium-connector-mysql-2.4.1.Final-plugin.tar.gz
tar -xvzf debezium-connector-postgres-2.4.1.Final-plugin.tar.gz
tar -xvzf debezium-connector-sqlserver-2.4.1.Final-plugin.tar.gz

'''edit file connect-distributed.properties yang ada di dalam folder /opt/confluent/confluent-7.5.2/etc/kafka'''
cd /opt/confluent/confluent-7.5.2/etc/kafka
vi connect-distributed.properties

'''tambahkan baris berikut di dalam file connect-distributed.properties'''
plugin.path=/usr/share/java,/opt/confluent/confluent-7.5.2/connectors

'''restart confluent'''
confluent local services restart

'''check confluent status'''
curl -XGET http://localhost:8083/connector-plugins | jq

'''maka akan muncul response seperti berikut ini:'''
[
  {
    "class": "io.debezium.connector.mysql.MySqlConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  {
    "class": "io.debezium.connector.postgresql.PostgresConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  {
    "class": "io.debezium.connector.sqlserver.SqlServerConnector",
    "type": "source",
    "version": "2.4.1.Final"
  },
  ...
]

'''artinya connector sudah terinstall dengan baik'''
'''dengan rangkaian langkah di atas, maka kafka sudah terinstall dengan baik'''
'''dan siap digunakan untuk keperluan streaming data'''

'''--------------------------------------------------------------------------------------'''